In [164]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [165]:
df = pd.read_csv('files/all.csv', index_col='Unnamed: 0')

In [166]:
# Dropping street and housenumber for now, but might be intereting later use it, eg via a graph library

# df = df.drop(['property.location.box', 'property.location.street',
#               'property.location.latitude', 'property.location.longitude',
#               'property.location.locality', 'property.location.number',
#               'Address',], axis=1)

df = df.drop(columns=['flags.main','customerLogoUrl','customerName','media.pictures',
                      'flags.secondary','Price','property.title','publication.size',
                      'publication.visualisationOption','External reference',
                      'Reference number of the EPC report','Website', 'Available as of'], axis =1)

In [167]:
# Removing outliers
df = df[df['Floor :'] < 30]

# Geen openbare verkopen waar we moeten bieden of verkopen op lijfrente.
df = df[df['price.label'] == 'Price']

# Geen slaapkamers vermeld is nogal raar.

df = df[df['property.bedroomCount'] > 0]

binary_cols = ['Double glazing','Armored door','Elevator','Basement',
             'Intercom','Planning permission obtained', 'Type of building']
df[binary_cols] =df[binary_cols].replace('Yes', 1).replace('No',0)


# Striping things like 'm²square meters' from numerical columns and
# converting them to float

numerical_cols = ['CO² emission', 'Living area','Living room surface',
            'Bedroom 1 surface','Bedroom 2 surface','Cadastral income',
            'E-level (primary energy consumption)','Terrace surface']

strippers = ['kg CO2/m²','kWh/m²kilowatt hour per square meters',
            'm²square meters','€']

for x in numerical_cols:
    for y in strippers:
        df[x] = df[x].str.strip(y).replace('Not specified',np.nan)
    df[x] = df[x].str.replace('.','').str.replace(',','.').replace('', np.nan)
    df[x] = df[x].astype(float)

# Dropping columns with duplicate information
to_drop = ['Floor','price.shortDisplayPrice','price.accessibilityPrice',
           'price.mainDisplayPrice', 'property.location.floor','Bedrooms','bedrooms','bedroom',
           'transaction.sale.price','bathroom','m²square meterslivable space']

df = df.drop(to_drop, axis=1)

In [168]:
for column in df.columns:
    if df[column].isnull().sum() > 0.8*len(df):
        df = df.drop(column, axis=1)
    elif len(df[column].unique()) == 1:
        df = df.drop(column, axis=1)

In [169]:
df.to_pickle('files/cleaned.pkl')